In [ ]:
# --- Install missing packages if necessary ---
# !pip install requests geopandas shapely

import requests
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import os

# --- User input ---
print("Enter the country (ISO3166-1 code, e.g., DE, FR, AT):")
country = input().strip().upper()

# --- Overpass API endpoint ---
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# --- Overpass QL query: fetch all wildlife crossings with geometry ---
query = f"""
[out:json][timeout:1800];
area["ISO3166-1"="{country}"][admin_level=2]->.searchArea;
(
  nwr["man_made"="wildlife_crossing"](area.searchArea);
  way["man_made"="wildlife_crossing"](area.searchArea);
  relation["man_made"="wildlife_crossing"](area.searchArea);
);
out body geom;
"""

# --- Send request ---
response = requests.get(OVERPASS_URL, params={"data": query})
response.raise_for_status()
data = response.json()

# --- Extract elements ---
elements = data["elements"]

# --- Build GeoDataFrame ---
features = []

for el in elements:
    tags = el.get("tags", {})

    # Node → Point
    if el["type"] == "node":
        geometry = Point(el["lon"], el["lat"])
        features.append({"id": el["id"], "geometry": geometry, **tags})

    # Way → LineString or Polygon
    elif el["type"] == "way" and "geometry" in el:
        coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
        if len(coords) >= 3 and coords[0] == coords[-1]:
            geometry = Polygon(coords)  # Closed → Polygon
        else:
            geometry = LineString(coords)  # Open → LineString
        features.append({"id": el["id"], "geometry": geometry, **tags})

    # Relation → MultiPolygon (if members have geometry)
    elif el["type"] == "relation" and "members" in el:
        polygons = []
        for member in el["members"]:
            if member["type"] == "way" and "geometry" in member:
                coords = [(pt["lon"], pt["lat"]) for pt in member["geometry"]]
                if len(coords) >= 3:
                    polygons.append(Polygon(coords))
        if polygons:
            geometry = MultiPolygon(polygons) if len(polygons) > 1 else polygons[0]
            features.append({"id": el["id"], "geometry": geometry, **tags})

# --- Create GeoDataFrame ---
if features:
    gdf = gpd.GeoDataFrame(features, geometry="geometry", crs="EPSG:4326")
else:
    gdf = gpd.GeoDataFrame(columns=["id", "geometry"])

# --- Filter by bridge = 'yes' if needed ---
if "bridge" in gdf.columns:
    gdf = gdf[gdf["bridge"] == "yes"]

# --- Save GeoJSON ---
output_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{country}_wildlife_crossings.geojson")
gdf.to_file(output_file, driver="GeoJSON")

print(f"GeoDataFrame contains {len(gdf)} features after filtering.")
print(f"Saved to {output_file}")
gdf.head()

Enter the country (ISO3166-1 code, e.g., DE, FR, AT):
GeoDataFrame contains 131 features after filtering.
Saved to c:\code\cassda-zertifikatsarbeit\data\DE_wildlife_crossings.geojson


,id,geometry,description,man_made,tunnel,material,name,layer,bridge,bridge:structure,...,construction,informal,mapillary,abandoned:highway,wikipedia,fence_type,wikimedia_commons,intermittent,fixme,type
6,32784047,"POLYGON ((10.67067 52.25805, 10.67117 52.25814...",Wildbrücke,wildlife_crossing,NaN,NaN,NaN,1,yes,beam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,32847260,"POLYGON ((11.52872 53.71641, 11.52841 53.71654...",NaN,wildlife_crossing,NaN,NaN,Grünbrücke Ahrensboek,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,34301551,"POLYGON ((10.71333 52.28455, 10.71331 52.28459...",Wildbrücke,wildlife_crossing,NaN,NaN,NaN,1,yes,beam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,60131669,"LINESTRING (14.18057 51.10918, 14.18087 51.10957)",NaN,wildlife_crossing,NaN,NaN,Haselmausbrücke,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,65243312,"POLYGON ((10.24894 53.99571, 10.24891 53.99571...",NaN,wildlife_crossing,NaN,NaN,Grünbrücke Kiebitzholm,1,yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# funktioniert noch nicht - weiss noch nicht genau wieso. Datenmenge? 
# in CH gibts nur Punkte zurück, obwohl auf der Webpage Punkte und Linien sind.
"""
# --- Install missing packages if necessary ---
# !pip install requests geopandas shapely

import requests
import geopandas as gpd
from shapely.geometry import LineString
import os

# --- User input for country ---
print("Please enter the country (ISO3166-1 code, e.g., DE, FR, AT):")
country = input().strip().upper()

# --- Overpass API endpoint ---
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# --- Overpass API query: motorways and trunks ---
query = f"""
[out:json][timeout:1800];
area["ISO3166-1"="{country}"][admin_level=2]->.searchArea;
(
  way["highway"="motorway"](area.searchArea);
  way["highway"="trunk"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# --- Send request to Overpass API ---
response = requests.get(OVERPASS_URL, params={'data': query})
response.raise_for_status()
data = response.json()

# --- Extract elements ---
elements = data['elements']

# --- Convert to GeoDataFrame (only LineStrings) ---
features = []
for el in elements:
    if el['type'] == 'way' and 'geometry' in el:
        coords = [(pt['lon'], pt['lat']) for pt in el['geometry']]
        # Only open ways → LineString
        if len(coords) >= 2 and coords[0] != coords[-1]:
            features.append({'id': el['id'], 'geometry': LineString(coords), **el.get('tags', {})})

# Check if any features were found
if not features:
    print("No LineString features found!")
    gdf = gpd.GeoDataFrame(columns=['id', 'geometry'])
else:
    gdf = gpd.GeoDataFrame(features, geometry='geometry', crs="EPSG:4326")
    print(f"Found {len(gdf)} LineString features.")

# --- Prepare output folder ---
output_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))
os.makedirs(output_dir, exist_ok=True)

# --- Save GeoJSON ---
output_file = os.path.join(output_dir, f"{country}_motorway_trunk_lines.geojson")
gdf.to_file(output_file, driver="GeoJSON")
print(f"Data saved to {output_file}")

# --- Display first rows ---
gdf.head()

"""

Please enter the country (ISO3166-1 code, e.g., DE, FR, AT):


 CH


No LineString features found!
Data saved to C:\tmp\CAS Spatial Data Analytics\Dokumentation\Projektarbeit\Datenverarbeitung\CASSDA_py\cassda-zertifikatsarbeit\data\CH_motorway_trunk_lines.geojson


C:\Users\HoeM\AppData\Local\miniconda3\Lib\site-packages\pyogrio\geopandas.py:710: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


,id,geometry


In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import LineString
import os

# --- User input for country ---
country = "CH"

# --- Overpass API endpoint ---
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# --- Overpass API query: motorways and trunks ---
query = f"""
[out:json][timeout:1800];
area["ISO3166-1"="{country}"][admin_level=2]->.searchArea;
(
  way["highway"="motorway"](area.searchArea);
  way["highway"="trunk"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# --- Send request to Overpass API ---
response = requests.get(OVERPASS_URL, params={"data": query})
response.raise_for_status()
data = response.json()

# --- Extract elements ---
elements = data["elements"]